# Face detection and recognition inference pipeline

The following example illustrates how to use the `facenet_pytorch` python package to perform face detection and recogition on an image dataset using an Inception Resnet V1 pretrained on the VGGFace2 dataset.

The following Pytorch methods are included:
* Datasets
* Dataloaders
* GPU/CPU processing

In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4

#### Determine if an nvidia GPU is available

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

Default params shown for illustration, but not needed. Note that, since MTCNN is a collection of neural nets and other code, the device must be passed in the following way to enable copying of objects when needed internally.

See `help(MTCNN)` for more details.

In [5]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device, keep_all = True
)

#### Define Inception Resnet V1 module

Set classify=True for pretrained classifier. For this example, we will use the model to output embeddings/CNN features. Note that for inference, it is important to set the model to `eval` mode.

See `help(InceptionResnetV1)` for more details.

In [6]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

#### Define a dataset and data loader

We add the `idx_to_class` attribute to the dataset to enable easy recoding of label indices to identity names later one.

In [7]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('../data/test_images')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
print(dataset.idx_to_class)
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

{0: 'angelina_jolie', 1: 'bradley_cooper', 2: 'kate_siegel', 3: 'obama', 4: 'paul_rudd', 5: 'shea_whigham'}


#### Perfom MTCNN facial detection

Iterate through the DataLoader object and detect faces and associated detection probabilities for each. The `MTCNN` forward method returns images cropped to the detected face, if a face was detected. By default only a single detected face is returned - to have `MTCNN` return all detected faces, set `keep_all=True` when creating the MTCNN object above.

To obtain bounding boxes rather than cropped face images, you can instead call the lower-level `mtcnn.detect()` function. See `help(mtcnn.detect)` for details.

In [11]:
aligned = []
names = []
for x, y in loader:
    # print(x, y) # <PIL.Image.IMage, index)
    x_aligned, probs = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(probs[0]))
        aligned.append(x_aligned[0].squeeze(0))
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.999983
Face detected with probability: 0.999934
Face detected with probability: 0.999733
Face detected with probability: 0.999975
Face detected with probability: 0.999880
Face detected with probability: 0.999992


#### Calculate image embeddings

MTCNN will return images of faces all the same size, enabling easy batch processing with the Resnet recognition module. Here, since we only have a few images, we build a single batch and perform inference on it. 

For real datasets, code should be modified to control batch sizes being passed to the Resnet, particularly if being processed on a GPU. For repeated testing, it is best to separate face detection (using MTCNN) from embedding or classification (using InceptionResnetV1), as calculation of cropped faces or bounding boxes can then be performed a single time and detected faces saved for future use.

In [12]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [17]:
from PIL import Image 
im = Image.open('../data/dest_images/findobama/twopeople.jpeg')
tmp_lst = []
ppls, probs = mtcnn(im, return_prob=True)
print(len(ppls))
if ppls is not None: 
		print('Face detected with probability: {:8f}'.format(probs[0]))
		# print(ppls)
ppls = ppls.to(device)
unknown_embeddings = resnet(ppls).detach().cpu()
# print(single_embeddings)

2
Face detected with probability: 1.000000


#### Print distance matrix for classes

In [16]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

                angelina_jolie  bradley_cooper  kate_siegel     obama  \
angelina_jolie        0.000000        1.447480     0.887728  1.507617   
bradley_cooper        1.447480        0.000000     1.313749  1.643433   
kate_siegel           0.887728        1.313749     0.000000  1.510497   
obama                 1.507617        1.643433     1.510497  0.000000   
paul_rudd             1.429847        1.013447     1.388377  1.527893   
shea_whigham          1.399073        1.038684     1.379655  1.636850   

                paul_rudd  shea_whigham  
angelina_jolie   1.429847      1.399073  
bradley_cooper   1.013447      1.038684  
kate_siegel      1.388377      1.379655  
obama            1.527893      1.636850  
paul_rudd        0.000000      1.100503  
shea_whigham     1.100503      0.000000  


In [18]:
for person in unknown_embeddings :
	for e1 in embeddings:
		print((person - e1).norm().item()) 
	

1.4332432746887207
1.6190237998962402
1.4941800832748413
0.5921740531921387
1.4947419166564941
1.5431840419769287
1.5635106563568115
1.4122507572174072
1.534064769744873
1.308187484741211
1.6113932132720947
1.4635746479034424
